In [1]:
import numpy as np
import pandas as pd
from math import isnan
from datetime import datetime,date
# Saved Tiingo API Token in 'token' variable in apiToken.py file
from apiToken import token

In [2]:
# Getting prices from Tiingo
def getPrice(token,ticker,startDate,endDate):
    return pd.read_csv('https://api.tiingo.com/tiingo/daily/'+ticker+'/prices?startDate='+startDate+'&endDate='+endDate+'&token='+token+'&format=csv')
today = str(date.today())
bndPrice = getPrice(token,'bnd','2019-09-30',today)
bndxPrice = getPrice(token,'bndx','2019-09-30',today)
vtiPrice = getPrice(token,'vti','2019-09-30',today)
vxusPrice = getPrice(token,'vxus','2019-09-30',today)
vtiPrice['returns'] = (vtiPrice.adjClose)/vtiPrice.adjClose.shift(1) - 1
vxusPrice['returns'] = (vxusPrice.adjClose)/vxusPrice.adjClose.shift(1) - 1
bndPrice['returns'] = (bndPrice.adjClose)/bndPrice.adjClose.shift(1) - 1
bndxPrice['returns'] = (bndxPrice.adjClose)/bndxPrice.adjClose.shift(1) - 1
bndPrice = bndPrice.loc[:,['date','returns','adjClose']]
bndxPrice = bndxPrice.loc[:,['date','returns','adjClose']]
vtiPrice = vtiPrice.loc[:,['date','returns','adjClose']]
vxusPrice = vxusPrice.loc[:,['date','returns','adjClose']]

In [3]:
# Reading saved data (Market cap is in billions)
vti = pd.read_csv('Data/vti_MC_Data.csv')
vxus = pd.read_csv('Data/vxus_MC_Data.csv')
bnd = pd.read_csv('Data/bnd_MC_Data.csv')
bndx = pd.read_csv('Data/bndx_MC_Data.csv')

In [4]:
df = vtiPrice.rename(columns={"returns":"VTI_Return","adjClose":"VTI_Price"}).merge(vxusPrice.rename(columns={"returns":"VXUS_Return","adjClose":"VXUS_Price"}),on='date',how='outer').merge(bndPrice.rename(columns={"returns":"BND_Return","adjClose":"BND_Price"}),on='date',how='outer').merge(bndxPrice.rename(columns={"returns":"BNDX_Return","adjClose":"BNDX_Price"}),on='date',how='outer').merge(vti.rename(columns={"Market Cap":"VTI_MarketCap"}),on='date',how='left').merge(vxus.rename(columns={"Market Cap":"VXUS_MarketCap"}),on='date',how='left').merge(bnd.rename(columns={"Market Cap":"BND_MarketCap"}),on='date',how='left').merge(bndx.rename(columns={"Market Cap":"BNDX_MarketCap"}),on='date',how='left').sort_values(by='date').reset_index(drop=True)

In [5]:
etfs = ['VTI','VXUS','BND','BNDX']
for etf in etfs:
    list = []
    i = 0
    value = 999
    for mc,ret in zip(df[etf+'_MarketCap'],df[etf+'_Return']):
        if i>0:
            value = (ret+1) * list[i-1]
        list.append(mc) if not isnan(mc) else list.append(value)
        i+=1
    df[etf+'_MarketCap'] = list

In [6]:
df['VTI_weight'] = df['VTI_MarketCap']/(df.VTI_MarketCap+df.VXUS_MarketCap+df.BND_MarketCap+df.BNDX_MarketCap)
df['VXUS_weight'] = df['VXUS_MarketCap']/(df.VTI_MarketCap+df.VXUS_MarketCap+df.BND_MarketCap+df.BNDX_MarketCap)
df['BND_weight'] = df['BND_MarketCap']/(df.VTI_MarketCap+df.VXUS_MarketCap+df.BND_MarketCap+df.BNDX_MarketCap)
df['BNDX_weight'] = df['BNDX_MarketCap']/(df.VTI_MarketCap+df.VXUS_MarketCap+df.BND_MarketCap+df.BNDX_MarketCap)

#Enter starting amount
startingAmount = 10000

portfolioVal = [0 for i in range(len(df))]
portfolioVal[0] = startingAmount
df['Portfolio_Value'] = portfolioVal

In [7]:
i=0
list2 = []
portfolioRet = 1
for vtiR,vxusR,bndR,bndxR,vtiW,vxusW,bndW,bndxW,portfolioValue in zip(df.VTI_Return,df.VXUS_Return,df.BND_Return,df.BNDX_Return,df.VTI_weight.shift(1),df.VXUS_weight.shift(1),df.BND_weight.shift(1),df.BNDX_weight.shift(1),df.Portfolio_Value):
    if i>0:
        portfolioRet = (vtiR*vtiW+vxusR*vxusW+bndR*bndW+bndxR*bndxW)+1
    list2.append(portfolioValue) if portfolioValue != 0 else list2.append(list2[i-1]*portfolioRet)
    i+=1
df['Portfolio_Value'] = list2

In [8]:
df

,date,VTI_Return,VTI_Price,VXUS_Return,VXUS_Price,BND_Return,BND_Price,BNDX_Return,BNDX_Price,VTI_MarketCap,VXUS_MarketCap,BND_MarketCap,BNDX_MarketCap,VTI_weight,VXUS_weight,BND_weight,BNDX_weight,Portfolio_Value
0,2019-09-30,NaN,148.817031,NaN,50.677485,NaN,82.542422,NaN,56.980979,29401.430000,23547.228000,22512.530000,19421.010000,0.309873,0.248173,0.237268,0.204685,10000.000000
1,2019-10-01,-0.013046,146.875511,-0.009098,50.216424,0.000967,82.622252,-0.001668,56.885962,29017.848430,23332.996541,22534.302867,19388.625144,0.307804,0.247503,0.239030,0.205663,9935.875746
2,2019-10-02,-0.016238,144.490496,-0.015628,49.431639,0.002253,82.808405,0.000000,56.885962,28546.646704,22968.347250,22585.073856,19388.625144,0.305349,0.245680,0.241581,0.207390,9853.133140
3,2019-10-03,0.008117,145.663292,0.007740,49.814222,0.003313,83.082734,0.003749,57.099236,28778.353338,23146.113779,22659.894262,19461.315968,0.306004,0.246116,0.240946,0.206935,9911.835869
4,2019-10-04,0.013058,147.565391,0.006696,50.147755,0.001651,83.219899,0.000170,57.108930,29154.146450,23301.089728,22697.304465,19464.620096,0.308127,0.246267,0.239886,0.205720,9972.066703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,2020-08-03,0.008700,166.960000,0.014450,51.950000,-0.000349,89.270000,-0.000215,58.170000,32499.167720,24290.505272,23804.044529,21036.126119,0.319780,0.239010,0.234223,0.206988,10678.033634
213,2020-08-04,0.003953,167.620000,0.007700,52.350000,0.002128,89.460000,0.002579,58.320000,32627.638316,24477.535149,23854.708453,21090.370900,0.319721,0.239858,0.233755,0.206667,10722.205141
214,2020-08-05,0.007278,168.840000,0.005922,52.660000,-0.001341,89.340000,-0.002229,58.190000,32865.114266,24622.483304,23822.710185,21043.358757,0.321094,0.240563,0.232749,0.205595,10754.084179
215,2020-08-06,0.004857,169.660000,0.003038,52.820000,0.001567,89.480000,0.001203,58.260000,33024.729249,24697.295255,23860.041498,21068.672988,0.321719,0.240595,0.232439,0.205246,10785.296959
